## Your Model 🌱Garden🌱 Execution Environment

Use this notebook to write a function that executes your model(s). Tag that function with the `@garden_pipeline` decorator.

Garden will take this notebook and build a container with it. When Garden executes your `@garden_pipeline`, it will be like like you have just run all the cells of this notebook once. So you can install libraries with `!pip install` and your function can use those libraries. You can also define helper functions and constants to use in your `@garden_pipeline`.

In [ ]:
from garden_ai.model_connectors import HFConnector
from garden_ai import PipelineMetadata, garden_pipeline

In [ ]:
import pandas as pd
import sklearn
import joblib

### Model connectors

Model connectors let Garden import metadata about your model.
They also have a `download` method that you can use to download your model weights.

In [ ]:
my_hugging_face_repo = HFConnector("garden-ai/sample_sklearn_model")

### Pipeline metadata


To publish your function, Garden needs metadata so that other users can discover it.
Edit this PipelineMetadata object to describe your function.


In [ ]:
my_pipeline_meta = PipelineMetadata(
    title="My Inference Function",
    description="Write a longer description here so that people know what your it does.",
    authors=["you", "your collaborator"],
    tags=["materials science", "your actual field"]
)

### Helper Functions

Define any helper functions you need and use them in the function you want to let people run remotely

In [ ]:
def preprocess(input_df: pd.DataFrame) -> pd.DataFrame:
    input_df.fillna(0, inplace=True)
    return input_df

### Write your pipeline function that will run remotely

The `@garden_pipeline` decorator makes this function available to run in your garden when you publish the notebook.
Download your model weights and call your model in this function.

In the decorator be sure to include:
- your pipeline metadata,
- connectors for any models you're using,
- the DOI of the garden you want this pipeline to be found in. (Check `garden-ai garden list` for the DOIs of your gardens.)

In [ ]:
@garden_pipeline(pipeline_meta=my_pipeline_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.23677/my-garden-doi")
def run_my_model(input_df: pd.DataFrame) -> pd.DataFrame:
    cleaned_df = preprocess(input_df)
    download_path = my_hugging_face_repo.download()
    model = joblib.load(f"{download_path}/model.pkl")
    return model.predict(cleaned_df)

### Test your pipeline function

Finally, make sure your `@garden_pipeline` works!
When Garden makes a container from your notebook, it runs all the cells in order and saves the notebook. Then users invoke your `@garden_pipeline` in the context of the notebook.

If you can hit "Kernel" -> "Restart and run all cells" and your test below works, your `@garden_pipeline` will work in your garden!


In [ ]:
# Replace with input that is relevant for your garden_pipeline
example_input = pd.DataFrame({
    'A': [1, 2, None, 4],
    'B': [None, 2, 3, 4],
    'C': [1, 2, 3, 4]
})
run_my_model(example_input)